<a href="https://colab.research.google.com/github/PedroTechy/CarrisInsight/blob/streaming_development/spark_jobs/extract_carris_vehicles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Authenticate with Google Cloud


In [2]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=koG8lBs5yV8Y80KHWJjux4YdqliSUA&prompt=consent&token_usage=remote&access_type=offline&code_challenge=jFo2BQ-gcLZTKiDJ8wEW5IgaxeHDt5NfTIxYMQwVPMo&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AanRRrs7R2-U1_ipAb-iFBRytzOhlX78w_gGubiDhNp0bsMgSbi61oEr0r1d9obj64l6sg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

# Step 2: Install Spark and BigQuery connector

In [3]:
!pip install pyspark

In [28]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, LongType
from pyspark.sql.functions import min, max, first, last, col, window, from_unixtime, to_timestamp, count, udf


In [35]:
spark = SparkSession.builder \
    .appName('pyspark-run-with-gcp-bucket') \
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile", "/content/.config/application_default_credentials.json")

# Step 3: Set environment variables for Spark and Java

# Step 4: Initialize Spark session

# Step 5: Define GCS input path and output BigQuery table

In [6]:
input_path = "gs://edit-de-project-streaming-data/carris-vehicles"
output_table = "data-eng-dev-437916.data_eng_project_group3_raw"

In [39]:
!rm -rf content/lake/

# Step 6: Read streaming data from GCS

In [34]:
# Define the schema for your JSON files

schema = StructType([
    StructField("bearing", FloatType(), True),
    StructField("block_id", StringType(), True),
    StructField("current_status", StringType(), True),
    StructField("id", StringType(), True),
    StructField("lat", FloatType(), True),
    StructField("line_id", StringType(), True),
    StructField("lon", FloatType(), True),
    StructField("pattern_id", StringType(), True),
    StructField("route_id", StringType(), True),
    StructField("schedule_relationship", StringType(), True),
    StructField("shift_id", StringType(), True),
    StructField("speed", FloatType(), True),
    StructField("stop_id", StringType(), True),
    StructField("timestamp", LongType(), True),
    StructField("trip_id", StringType(), True)
])

In [41]:

def transform_data(df, batch_id):
  transformed_df = df.withColumn("timestamp", to_timestamp(from_unixtime("timestamp")))
  print(transformed_df.orderBy("timestamp", ascending=False).show(5))
  print(transformed_df.orderBy("timestamp", ascending=True).show(5))
  # Write a df with the datetype transform only
  transformed_df.write.mode("append").format("parquet").save("content/lake/processing/vehicles/tests")

  window_spec = window("timestamp", "2 minutes")
  # Group by vehicle ID and window, then get the first and last timestamps and lat/lon values
  result_df = (
      transformed_df.groupBy("id", window_spec)
      .agg(
          max("current_status").alias("current_status"),
          max("route_id").alias("route_id"),
          max("trip_id").alias("count"),
          count("trip_id").alias("count"),
          min("timestamp").alias("first_timestamp"),
          max("timestamp").alias("last_timestamp"),
          first("lat").alias("first_lat"),
          first("lon").alias("first_lon"),
          last("lat").alias("last_lat"),
          last("lon").alias("last_lon")
      )

      #.orderBy("last_timestamp", ascending=False)#.dropDuplicates(["id"])
      )
  print(result_df.orderBy("first_timestamp", ascending=False).show(5))
  print(result_df.orderBy("last_timestamp", ascending=True).show(5))

  result_df.write.mode("append").format("parquet").save("content/lake/processing/vehicles/data")
  print(result_df.count())
  return result_df



In [42]:
import math
def haversine(lat1, lon1, lat2, lon2):
    try:
      # Earth radius in kilometers
      R = 6371.0

      # Convert latitude and longitude from degrees to radians
      lat1_rad, lon1_rad = math.radians(lat1), math.radians(lon1)
      lat2_rad, lon2_rad = math.radians(lat2), math.radians(lon2)

      # Differences
      delta_lat = lat2_rad - lat1_rad
      delta_lon = lon2_rad - lon1_rad

      # Haversine formula
      a = math.sin(delta_lat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2)**2
      c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

      # Distance
      distance = R * c
      return distance
    except:
      return 0




haversine_udf = udf(haversine, FloatType())



In [ ]:
df = (spark.readStream.option("maxFilesPerTrigger", 1)
    .format("json")
    .schema(schema)
    .load(input_path))
print("will start streaming")


transformed_df = df.withColumn("timestamp", to_timestamp(from_unixtime("timestamp")))
# Write a df with the datetype transform only

window_spec = window("timestamp", "2 minutes", "10 seconds")
# Group by vehicle ID and window, then get the first and last timestamps and lat/lon values
result_df = (
    transformed_df
        .withWatermark("timestamp", "3 minutes")
    .groupBy("id", "trip_id", window_spec)
    .agg(
        max("current_status").alias("current_status"),
        max("route_id").alias("route_id"),
        min("timestamp").alias("first_timestamp"),
        max("timestamp").alias("last_timestamp"),
        first("lat").alias("first_lat"),
        first("lon").alias("first_lon"),
        last("lat").alias("last_lat"),
        last("lon").alias("last_lon")
    ).withColumn(
        "distance",
        haversine_udf(col("first_lat"), col("first_lon"), col("last_lat"), col("last_lon")
        )
        #.orderBy("last_timestamp", ascending=False)#.dropDuplicates(["id"])
    ).withColumn(
        "time_delta", col("last_timestamp").cast("long") - col("first_timestamp").cast("long")
    ).withColumn("average_speed", col("distance") / (col("time_delta") / 3600))
    #.orderBy("last_timestamp", ascending=False)
)

query = (result_df.writeStream
.outputMode('append')
.option('checkpointLocation', 'content/lake/processing/vehicles_checkpoint') #using a common checkpoint for both messages streams.
.trigger(processingTime='10 seconds')
#.foreach(transform_data)
.start('content/lake/processing/vehicles/data')
)

query.awaitTermination(300)

query.stop()


will start streaming


In [21]:
df = spark.read.format("parquet").load("content/lake/processing/vehicles/data")


In [67]:
tests = spark.read.format("parquet").load("content/lake/processing/vehicles/test/data")


In [23]:
df.show()

+--------+--------------------+-----+-------------------+-------------------+---------+---------+---------+---------+
|      id|              window|count|    first_timestamp|     last_timestamp|first_lat|first_lon| last_lat| last_lon|
+--------+--------------------+-----+-------------------+-------------------+---------+---------+---------+---------+
| 42|2319|{2025-01-17 19:15...|    3|2025-01-17 19:16:28|2025-01-17 19:17:09| 38.75608|-9.151774|38.759094|-9.154597|
| 41|1333|{2025-01-17 19:15...|    3|2025-01-17 19:16:30|2025-01-17 19:17:45| 38.73912|-9.304881|38.744076|-9.303941|
| 41|1246|{2025-01-17 19:16...|    4|2025-01-17 19:16:32|2025-01-17 19:17:50|38.766533|-9.206014|38.765526|-9.205146|
| 41|1413|{2025-01-17 19:16...|    3|2025-01-17 19:16:47|2025-01-17 19:17:49| 38.75795|-9.382061|38.759228|-9.375238|
|  41|745|{2025-01-17 19:16...|    4|2025-01-17 19:16:25|2025-01-17 19:17:33| 38.86975|-9.354175|38.864723|-9.356451|
| 42|2581|{2025-01-17 19:16...|    3|2025-01-17 19:16:39

In [25]:

df.filter(df["id"] == '42|2319').show()

+-------+--------------------+-----+-------------------+-------------------+---------+---------+---------+---------+
|     id|              window|count|    first_timestamp|     last_timestamp|first_lat|first_lon| last_lat| last_lon|
+-------+--------------------+-----+-------------------+-------------------+---------+---------+---------+---------+
|42|2319|{2025-01-17 19:15...|    3|2025-01-17 19:16:28|2025-01-17 19:17:09| 38.75608|-9.151774|38.759094|-9.154597|
|42|2319|{2025-01-17 19:16...|    4|2025-01-17 19:16:28|2025-01-17 19:17:51| 38.75608|-9.151774|38.759094|-9.154597|
|42|2319|{2025-01-17 19:16...|    4|2025-01-17 19:16:28|2025-01-17 19:17:51| 38.75608|-9.151774|38.759094|-9.154597|
|42|2319|{2025-01-17 19:16...|    4|2025-01-17 19:17:09|2025-01-17 19:18:27|38.759094|-9.154597|38.759506|-9.156824|
|42|2319|{2025-01-17 19:15...|    3|2025-01-17 19:16:28|2025-01-17 19:17:09| 38.75608|-9.151774|38.759094|-9.154597|
|42|2319|{2025-01-17 19:15...|    3|2025-01-17 19:16:28|2025-01-

In [26]:
tests.filter(tests["id"] == '44|12097')

NameError: name 'tests' is not defined

In [58]:
# prompt: create a new column that applies haversine to each row to the values of columns first_lat, first_lon, last_lat and last_lon

from pyspark.sql.functions import udf
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType,   StringType, IntegerType


haversine_udf = F.udf(haversine, FloatType())

df = df.withColumn("distance", haversine_udf(F.col("first_lat"), F.col("first_lon"), F.col("last_lat"), F.col("last_lon")))

# Apply the UDF to create the new column


In [59]:
window_spec = window("timestamp", "2 minutes")
  # Group by vehicle ID and window, then get the first and last timestamps and lat/lon values
result_df = (tests.groupBy("id", window_spec)
      .agg(
          min("timestamp").alias("first_timestamp"),
          max("timestamp").alias("last_timestamp"),
          first("lat").alias("first_lat"),
          first("lon").alias("first_lon"),
          last("lat").alias("last_lat"),
          last("lon").alias("last_lon")
      ).orderBy("last_timestamp", ascending=False).dropDuplicates(["id"]).withColumn("distance", haversine_udf(F.col("first_lat"), F.col("first_lon"), F.col("last_lat"), F.col("last_lon")))
      )

In [23]:
df.filter(df["first_lat"] != df["last_lat"]).show()

+---+------+---------------+--------------+---------+---------+--------+--------+
| id|window|first_timestamp|last_timestamp|first_lat|first_lon|last_lat|last_lon|
+---+------+---------------+--------------+---------+---------+--------+--------+
+---+------+---------------+--------------+---------+---------+--------+--------+



In [61]:
result_df.filter(result_df["first_lat"] != result_df["last_lat"]).show()

+-------+--------------------+-------------------+-------------------+---------+---------+---------+---------+-----------+
|     id|              window|    first_timestamp|     last_timestamp|first_lat|first_lon| last_lat| last_lon|   distance|
+-------+--------------------+-------------------+-------------------+---------+---------+---------+---------+-----------+
|41|1100|{2025-01-17 08:48...|2025-01-17 08:48:02|2025-01-17 08:48:25|38.721485|-9.202832|38.722572|-9.199898|   0.281819|
|41|1102|{2025-01-17 10:16...|2025-01-17 10:16:22|2025-01-17 10:17:11|38.725082|-9.249238|38.723186|-9.248443| 0.22181912|
|41|1103|{2025-01-17 10:16...|2025-01-17 10:16:19|2025-01-17 10:17:10|38.728683|-9.245057|38.725952|-9.243215|  0.3431985|
|41|1104|{2025-01-17 10:16...|2025-01-17 10:16:16|2025-01-17 10:17:15|38.728886|-9.244875|38.726624| -9.24444|  0.2543488|
|41|1105|{2025-01-17 10:16...|2025-01-17 10:16:14|2025-01-17 10:16:57|38.734478|-9.305613|38.734356|-9.306508| 0.07885109|
|41|1107|{2025-0

In [ ]:
# prompt: convert the column timestamp to timestamp

from pyspark.sql.functions import from_unixtime, to_timestamp


In [ ]:
# prompt: group by vehicle in 2 minutes buckets

from pyspark.sql.functions import window, from_unixtime, col, to_timestamp

# Assuming 'vehicles_table' is your table with the vehicle data

# Read the data
vehicle_data = spark.read.parquet("vehicles_table")

# Convert the timestamp to a timestamp type and adjust timezone
vehicle_data = vehicle_data.withColumn("timestamp_readable", to_timestamp(from_unixtime(col("timestamp") / 1000)))

# Group by vehicle and 2-minute intervals
grouped_vehicle_data = vehicle_data.groupBy(
    "id", window("timestamp_readable", "2 minutes")
).count()


# Show the results
grouped_vehicle_data.show(truncate=False)

# Optional: Write the grouped data to a new location
# grouped_vehicle_data.write.mode("overwrite").parquet("grouped_vehicles_data")

In [ ]:
  spark.read.table("vehicles_table").withColumn("timestamp", to_timestamp(from_unixtime("timestamp"))).show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|  183.0|20250116-64010165...| IN_TRANSIT_TO|44|12745| 38.76761|   4720|-9.100393|  4720_0_1|  4720_0|            SCHEDULED|113260234560|      0.0| 060011|2025-01-16 11:16:53|4720_0_1|2700|111...|
|   58.0|20250116-64010273...| IN_TRANSIT_TO|44|12557| 38.57138|   4641| -9.03899|  4641_0_2|  4641_0|            SCHEDULED|111500234560|      0.0| 150013|2025-01-16 11:16:43|4641_0_2|2700|102...|
|   12.0|202501

# Step 7: Write streaming data to BigQuery with auto-table creation

In [ ]:
streaming_query.stop()